#Preparing data for input into the hmm["activity"] models

In [2]:
from __future__ import division

%matplotlib inline
import pandas as pd
import thinkdsp
import thinkplot
import numpy as np

from magnitude import magnitude
from pipeline import preprocess, extract_features_with_sliding_window, learn

ImportError: No module named hmmlearn.hmm

In [ ]:
#Ryan's fancy way of inputting data in an easier way
data_dict = {'walking':{},'jogging':{},'upstairs':{},'downstairs':{}}
names = ['meg','ryan','dennis']
acts = ['walking', 'jogging', 'upstairs', 'downstairs']
for name in names:
    data_file_names = ['data/{}_{}_long.csv'.format(name, activity) for activity in acts]
    for i,file in enumerate(data_file_names):
        df = pd.read_csv(file)
        data_dict[acts[i]][name] = df

In [ ]:
data_dict["walking"]["meg"].head()

In [ ]:
feature_dict = {}
for activity, activity_data_dict in data_dict.iteritems():
    print "Activity: {}".format(activity)
    feature_dict[activity] = {}
    for person, person_data in activity_data_dict.iteritems():
        print "Person: {}".format(person)
        print "Person Data: \n {}".format(person_data.head())
        
        a_norm = preprocess(person_data)
        obs = extract_features_with_sliding_window(a_norm, n_windows=10)
        feature_dict[activity][person] = obs
    print

Here is the matrix that represents the features extracted for a sequence of windows.  Let's call this matrix $X$

In [ ]:
feature_dict['downstairs']['dennis']

Check out the ```learn``` function in ```pipeline.py```: the real magic takes place there.  After training on the features, a dictionary of hidden markov models is returned (```hidden_markov_models```).  These four models can give us a (log)likelihood that a new sequence belongs to the activity they model. The model with the maximum likelihood will tell us which activity is happening.

In [ ]:
hidden_markov_models = learn(feature_dict)

Plug in different strings for the feature dict, to make a score of how likely the sequence, for a particular activity and user, be represented by the activity model.  Right now, we are training and testing on the same dataset.

In [ ]:
max_val = 0
max_act = "none"
for activity in acts:
    print activity
    predicted_score = hidden_markov_models[activity].score(feature_dict['downstairs']['dennis'][0])
    print predicted_score
    if predicted_score > max_val:
        max_val = predicted_score
        max_act = activity

In [ ]:
print("The predicted activity is: %s"+ max_act)

If we want to make activity prediction on unseen data, all we have to do is the following:
    
    for activity in acts:
        print activity
        
        # Score a unseen sequence of observations
        # unseen_obs will be shape (n_windows, n_features)
        # unseen_obs = feature_dict[activity][unseen_user][i]
        print hidden_markov_model[activity].score(unseen_obs)

Then check out which log likelihood is highest.  That will be the predicted activity.
        